# US Air Freight and Waterway Volumes

---

A look at air and domestic waterway freight volumes, according to the US DOT Bureau of Transportation Statistics.

In [1]:
import pandas as pd
import altair as alt
from os import environ
import re

if environ.get("GITHUB_WORKFLOW"):
    raise RuntimeError("Requires manual updates")

In [2]:
# https://www.transtats.bts.gov/osea/seasonaladjustment/

af_res = !curl -k 'https://transtats.bts.gov/osea/seasonaladjustment/?PageVar=AIR_RTMFM'

#print("\n".join(af_res))

af_raw = [r for r in af_res if "series:" in r]

df_af = pd.DataFrame.from_records(re.findall("'(\d\d/\d\d/\d\d\d\d \d\d:\d\d:\d\d)'\), y: (\d+)", "".join(af_raw)),
                                  columns=["date", "tonMiles"])

df_af = df_af.dropna()

df_af['dt'] = df_af.date.apply(pd.to_datetime)

#df_af.head()

## US Airline Freight

In [3]:
alt.Chart(df_af).mark_line().encode(
    alt.X('dt:T', axis=alt.Axis(title='')),
    alt.Y('tonMiles:Q', axis=alt.Axis(title='Volume [ton-miles]'))
).properties(
    title='US Air Freight Volume',
    width=700,
    height=400
)

alt.Chart(...)

In [4]:
df_af['change'] = df_af.set_index('dt')['tonMiles'].apply(int).pct_change(12).apply(lambda v: v * 100).values

c = alt.Chart(df_af).mark_bar(size=2).encode(
    alt.X('dt:T', axis=alt.Axis(title='')),
    alt.Y('change:Q', axis=alt.Axis(title='Year-over-Year Change [%]')),
    tooltip=[alt.Tooltip('dt:T', title='Date'), alt.Tooltip('change:Q', title='% Change', format=',.01f')],
    color=alt.condition(f"datum.change < 0",
        alt.value('lightsalmon'),
        alt.value('royalblue')
    ),
).properties(
    title='US Air Freight Volume Growth',
    width=750,
    height=400
)

#c.save('transportation-air-waterway.png')
c.display()

alt.Chart(...)

## Internal US Waterways Shipping

In [5]:
wwf_res = !curl -k 'https://transtats.bts.gov/osea/seasonaladjustment/?PageVar=WATERBORNE'

wwf_raw = [r for r in wwf_res if "series:" in r]


df_wwf = pd.DataFrame.from_records(re.findall("'(\d\d/\d\d/\d\d\d\d \d\d:\d\d:\d\d)'\), y: (\d+)", "".join(wwf_raw)),
                                  columns=["date", "megaShortTons"])

df_wwf = df_wwf.dropna()

df_wwf['dt'] = df_wwf.date.apply(pd.to_datetime)

#df_wwf.head()

In [6]:
alt.Chart(df_wwf).mark_line().encode(
    alt.X('dt:T', axis=alt.Axis(title='')),
    alt.Y('megaShortTons:Q', axis=alt.Axis(title='Volume [Million short tons]'))
).properties(
    title='US Internal Waterways Shipping Volume',
    width=700,
    height=400
)

alt.Chart(...)

In [7]:
df_wwf['change'] = df_wwf.set_index('dt')['megaShortTons'].apply(int).pct_change(12).apply(lambda v: v * 100).values

alt.Chart(df_wwf).mark_bar(size=2).encode(
    alt.X('dt:T', axis=alt.Axis(title='')),
    alt.Y('change:Q', axis=alt.Axis(title='Year-over-Year Change [%]')),
    tooltip=[alt.Tooltip('dt:T', title='Date'), alt.Tooltip('change:Q', title='% Change', format=',.01f')],
    color=alt.condition(f"datum.change < 0",
        alt.value('lightsalmon'),
        alt.value('royalblue')
    ),
).properties(
    title='US Internal Waterways Volume Growth',
    width=750,
    height=400
)

alt.Chart(...)